In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
num_lag = 2
num_lead = 2

In [3]:
o2_dir = Path('../data/daily_data/o2_raw')
o2_files = sorted(os.listdir(o2_dir))
o2_files = [file for file in o2_files if file[0] != '.']
o2_dfs = [pd.read_csv(o2_dir / file) for file in o2_files]
o2_df = pd.concat(o2_dfs, axis=0)
o2_df.drop('Unnamed: 0', inplace=True, axis=1)

geo_dir = Path('../data/daily_data/geo_raw')
geo_files = sorted(os.listdir(geo_dir))
geo_files = [file for file in geo_files if file[0] != '.']
geo_files.remove('readme.txt')
geo_dfs = [pd.read_csv(geo_dir / file) for file in geo_files]
geo_df = pd.concat(geo_dfs, axis=0)

geo_df['lon_rounded_up'] = 5 * geo_df['longitude_bins'] - 95
geo_df['lat_rounded_up'] = 5 * geo_df['latitude_bins'] - 10
geo_df.drop(['longitude_bins', 'latitude_bins'], axis=1, inplace=True)
geo_df = geo_df[geo_df.time <= '2024-03-31']
geo_df.rename({'time': 'date'}, axis=1, inplace=True)

o2_df = o2_df[o2_df.date >= '2021-11-30']

In [4]:
sal_dir = Path('../data/daily_data/Salinity_data')
sal_files = sorted(os.listdir(sal_dir))
sal_files = [file for file in sal_files if file[0] != '.']

In [19]:
sal_dfs = [pd.read_csv(sal_dir / file) for file in sal_files]

In [23]:
for df in sal_dfs:
    df.rename({
        'Salinity':'salinity'
    }, inplace=True, axis=1)

In [24]:
sal_df = pd.concat(sal_dfs, axis=0)

In [25]:
sal_df

,longitude,latitude,time,depth,salinity
0,-100.0,-15.0,2021-11-01,0.505760,35.627200
1,-40.0,35.0,2021-11-01,22.757616,36.542046
2,-40.0,35.0,2021-11-01,26.558300,36.544285
3,-40.0,35.0,2021-11-01,30.874561,36.547882
4,-40.0,35.0,2021-11-01,35.740204,36.553055
...,...,...,...,...,...
465397,0.0,35.0,2023-01-01,25.211410,NaN
465398,0.0,35.0,2023-01-01,29.444730,NaN
465399,0.0,35.0,2023-01-01,34.434150,NaN
465400,0.0,35.0,2023-01-01,40.344050,NaN


In [26]:
o2_df

,date,lon_rounded_up,lat_rounded_up,depth,o2
0,2021-11-30,-95,-10,0.494025,224.929197
1,2021-11-30,-95,-10,1.541375,224.940039
2,2021-11-30,-95,-10,2.645669,224.949801
3,2021-11-30,-95,-10,3.819495,224.959797
4,2021-11-30,-95,-10,5.078224,224.969852
...,...,...,...,...,...
3595,2024-03-31,0,35,25.211409,NaN
3596,2024-03-31,0,35,29.444729,NaN
3597,2024-03-31,0,35,34.434155,NaN
3598,2024-03-31,0,35,40.344050,NaN


In [27]:
geo_df

,date,depth,chl,no3,po4,si,lon_rounded_up,lat_rounded_up
0,2021-11-30,0.494025,0.149859,2.325064,0.826761,3.878553,-95,-10
1,2021-11-30,1.541375,0.149860,2.325051,0.826756,3.878534,-95,-10
2,2021-11-30,2.645669,0.149862,2.325067,0.826752,3.878516,-95,-10
3,2021-11-30,3.819495,0.149865,2.325118,0.826746,3.878498,-95,-10
4,2021-11-30,5.078224,0.149870,2.325200,0.826739,3.878478,-95,-10
...,...,...,...,...,...,...,...,...
661297,2024-03-31,25.211409,NaN,NaN,NaN,NaN,0,35
661298,2024-03-31,29.444729,NaN,NaN,NaN,NaN,0,35
661299,2024-03-31,34.434155,NaN,NaN,NaN,NaN,0,35
661300,2024-03-31,40.344050,NaN,NaN,NaN,NaN,0,35


In [28]:
o2_df = o2_df.groupby(
    by=['date', 'lon_rounded_up', 'lat_rounded_up']).mean()
o2_df.reset_index(inplace=True)
o2_df.drop('depth', axis=1, inplace=True)

In [29]:
o2_df

,date,lon_rounded_up,lat_rounded_up,o2
0,2021-11-30,-95,-10,225.033174
1,2021-11-30,-95,-5,221.951588
2,2021-11-30,-95,0,185.608996
3,2021-11-30,-95,5,191.597901
4,2021-11-30,-95,10,183.150434
...,...,...,...,...
170595,2024-03-31,0,15,NaN
170596,2024-03-31,0,20,NaN
170597,2024-03-31,0,25,NaN
170598,2024-03-31,0,30,NaN


In [30]:
geo_df = geo_df.groupby(
    by=['date', 'lon_rounded_up', 'lat_rounded_up']).mean()
geo_df.reset_index(inplace=True)
geo_df.drop('depth', axis=1, inplace=True)

In [31]:
geo_df

,date,lon_rounded_up,lat_rounded_up,chl,no3,po4,si
0,2021-11-30,-95,-10,0.149922,2.320183,0.825861,3.871832
1,2021-11-30,-95,-5,0.263287,6.655004,1.011894,4.183797
2,2021-11-30,-95,0,0.497302,11.343047,1.202757,8.277502
3,2021-11-30,-95,5,0.319714,5.631621,0.752967,5.382875
4,2021-11-30,-95,10,0.267116,5.082589,0.671990,4.205572
...,...,...,...,...,...,...,...
170595,2024-03-31,0,15,NaN,NaN,NaN,NaN
170596,2024-03-31,0,20,NaN,NaN,NaN,NaN
170597,2024-03-31,0,25,NaN,NaN,NaN,NaN
170598,2024-03-31,0,30,NaN,NaN,NaN,NaN


In [32]:
sal_df

,longitude,latitude,time,depth,salinity
0,-100.0,-15.0,2021-11-01,0.505760,35.627200
1,-40.0,35.0,2021-11-01,22.757616,36.542046
2,-40.0,35.0,2021-11-01,26.558300,36.544285
3,-40.0,35.0,2021-11-01,30.874561,36.547882
4,-40.0,35.0,2021-11-01,35.740204,36.553055
...,...,...,...,...,...
465397,0.0,35.0,2023-01-01,25.211410,NaN
465398,0.0,35.0,2023-01-01,29.444730,NaN
465399,0.0,35.0,2023-01-01,34.434150,NaN
465400,0.0,35.0,2023-01-01,40.344050,NaN


In [35]:
sal_df.rename({
    'time':'date'
}, inplace=True, axis=1)

In [37]:
sal_df[sal_df.longitude != 0.0]

,longitude,latitude,date,depth,salinity
0,-100.0,-15.0,2021-11-01,0.505760,35.627200
1,-40.0,35.0,2021-11-01,22.757616,36.542046
2,-40.0,35.0,2021-11-01,26.558300,36.544285
3,-40.0,35.0,2021-11-01,30.874561,36.547882
4,-40.0,35.0,2021-11-01,35.740204,36.553055
...,...,...,...,...,...
422827,-5.0,35.0,2023-01-01,25.211410,NaN
422828,-5.0,35.0,2023-01-01,29.444730,NaN
422829,-5.0,35.0,2023-01-01,34.434150,NaN
422830,-5.0,35.0,2023-01-01,40.344050,NaN


In [40]:
sal_df = sal_df[(sal_df.longitude != 0.0) & (sal_df.latitude != 35.0)]

In [41]:
sal_df

,longitude,latitude,date,depth,salinity
0,-100.0,-15.0,2021-11-01,0.505760,35.6272
7,-10.0,15.0,2021-11-01,47.211895,NaN
8,-10.0,15.0,2021-11-01,41.180023,NaN
9,-10.0,15.0,2021-11-01,35.740204,NaN
10,-10.0,15.0,2021-11-01,30.874561,NaN
...,...,...,...,...,...
418957,-5.0,30.0,2023-01-01,25.211410,NaN
418958,-5.0,30.0,2023-01-01,29.444730,NaN
418959,-5.0,30.0,2023-01-01,34.434150,NaN
418960,-5.0,30.0,2023-01-01,40.344050,NaN


In [42]:
pd.set_option('display.max_rows', 500)
sal_df.head(500)

,longitude,latitude,date,depth,salinity
0,-100.0,-15.0,2021-11-01,0.505760,35.627200
7,-10.0,15.0,2021-11-01,47.211895,NaN
8,-10.0,15.0,2021-11-01,41.180023,NaN
9,-10.0,15.0,2021-11-01,35.740204,NaN
10,-10.0,15.0,2021-11-01,30.874561,NaN
11,-10.0,15.0,2021-11-01,26.558300,NaN
12,-10.0,15.0,2021-11-01,22.757616,NaN
13,-10.0,15.0,2021-11-01,19.429802,NaN
14,-10.0,15.0,2021-11-01,16.525322,NaN
15,-10.0,15.0,2021-11-01,13.991038,NaN


In [43]:
sal_df

,longitude,latitude,date,depth,salinity
0,-100.0,-15.0,2021-11-01,0.505760,35.6272
7,-10.0,15.0,2021-11-01,47.211895,NaN
8,-10.0,15.0,2021-11-01,41.180023,NaN
9,-10.0,15.0,2021-11-01,35.740204,NaN
10,-10.0,15.0,2021-11-01,30.874561,NaN
...,...,...,...,...,...
418957,-5.0,30.0,2023-01-01,25.211410,NaN
418958,-5.0,30.0,2023-01-01,29.444730,NaN
418959,-5.0,30.0,2023-01-01,34.434150,NaN
418960,-5.0,30.0,2023-01-01,40.344050,NaN


In [44]:
sal_df.longitude = (sal_df.longitude + 5).astype(int)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_25791/482266998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sal_df.longitude = (sal_df.longitude + 5).astype(int)


In [45]:
sal_df

,longitude,latitude,date,depth,salinity
0,-95,-15.0,2021-11-01,0.505760,35.6272
7,-5,15.0,2021-11-01,47.211895,NaN
8,-5,15.0,2021-11-01,41.180023,NaN
9,-5,15.0,2021-11-01,35.740204,NaN
10,-5,15.0,2021-11-01,30.874561,NaN
...,...,...,...,...,...
418957,0,30.0,2023-01-01,25.211410,NaN
418958,0,30.0,2023-01-01,29.444730,NaN
418959,0,30.0,2023-01-01,34.434150,NaN
418960,0,30.0,2023-01-01,40.344050,NaN


In [46]:
sal_df.latitude = (sal_df.latitude + 5).astype(int)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_25791/3743474225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sal_df.latitude = (sal_df.latitude + 5).astype(int)


In [47]:
sal_df.rename({
    'longitude': 'lon_rounded_up',
    'latitude': 'lat_rounded_up'
}, inplace=True, axis=1)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_25791/4050519258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sal_df.rename({


In [48]:
sal_df

,lon_rounded_up,lat_rounded_up,date,depth,salinity
0,-95,-10,2021-11-01,0.505760,35.6272
7,-5,20,2021-11-01,47.211895,NaN
8,-5,20,2021-11-01,41.180023,NaN
9,-5,20,2021-11-01,35.740204,NaN
10,-5,20,2021-11-01,30.874561,NaN
...,...,...,...,...,...
418957,0,35,2023-01-01,25.211410,NaN
418958,0,35,2023-01-01,29.444730,NaN
418959,0,35,2023-01-01,34.434150,NaN
418960,0,35,2023-01-01,40.344050,NaN


In [49]:
sal_df = sal_df.groupby(
    by=['date', 'lon_rounded_up', 'lat_rounded_up']).mean()
sal_df.reset_index(inplace=True)
sal_df.drop('depth', axis=1, inplace=True)

In [50]:
sal_df

,date,lon_rounded_up,lat_rounded_up,salinity
0,2021-11-01,-95,-10,35.626152
1,2021-11-01,-95,-5,35.124233
2,2021-11-01,-95,0,34.728901
3,2021-11-01,-95,5,33.597405
4,2021-11-01,-95,10,33.319750
...,...,...,...,...
205795,2024-08-25,0,15,NaN
205796,2024-08-25,0,20,NaN
205797,2024-08-25,0,25,NaN
205798,2024-08-25,0,30,NaN


In [51]:
sal_df = sal_df[(sal_df.date >= '2021-11-30') & (sal_df.date <= '2024-03-31')]

In [52]:
sal_df

,date,lon_rounded_up,lat_rounded_up,salinity
5800,2021-11-30,-95,-10,35.653211
5801,2021-11-30,-95,-5,35.205782
5802,2021-11-30,-95,0,34.758569
5803,2021-11-30,-95,5,33.841740
5804,2021-11-30,-95,10,33.260691
...,...,...,...,...
176395,2024-03-31,0,15,NaN
176396,2024-03-31,0,20,NaN
176397,2024-03-31,0,25,NaN
176398,2024-03-31,0,30,NaN


In [53]:
temp_dir = Path('../data/daily_data/Temp')
temp_files = sorted(os.listdir(temp_dir))
temp_files = [file for file in temp_files if file[0] != '.']

In [54]:
temp_dfs = [pd.read_csv(temp_dir / file) for file in temp_files]

In [57]:
for df in temp_dfs:
    df.rename({
        'Temp': 'temp'
    }, inplace=True, axis=1)

In [68]:
temp_df = pd.concat(temp_dfs, axis=0)

In [69]:
temp_df

,longitude,latitude,time,depth,temp
0,-100.0,-15.0,2021-11-01,0.505760,21.621576
1,-100.0,-15.0,2021-11-01,1.555855,21.619543
2,-100.0,-15.0,2021-11-01,2.667682,21.614853
3,-100.0,-15.0,2021-11-01,3.856280,21.608278
4,-100.0,-15.0,2021-11-01,5.140361,21.601225
...,...,...,...,...,...
477871,-80.0,-15.0,2024-08-25,7.929560,15.865350
477872,-80.0,-15.0,2024-08-25,6.440614,15.949759
477873,-80.0,-15.0,2024-08-25,5.078224,15.973435
477874,-100.0,35.0,2024-08-25,47.373690,NaN


In [70]:
temp_df = temp_df[(temp_df.longitude != 0.0) & (temp_df.latitude != 35.0)]

In [71]:
temp_df.longitude = (temp_df.longitude + 5).astype(int)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_25791/887504507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.longitude = (temp_df.longitude + 5).astype(int)


In [72]:
temp_df.latitude = (temp_df.latitude + 5).astype(int)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_25791/2709105923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.latitude = (temp_df.latitude + 5).astype(int)


In [73]:
temp_df.rename({
    'longitude': 'lon_rounded_up',
    'latitude': 'lat_rounded_up',
    'time': 'date'
}, inplace=True, axis=1)

/var/folders/h9/7qb3ygvs0mzb0cdjbh8z8svr0000gn/T/ipykernel_25791/4253048819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.rename({


In [74]:
temp_df

,lon_rounded_up,lat_rounded_up,date,depth,temp
0,-95,-10,2021-11-01,0.505760,21.621576
1,-95,-10,2021-11-01,1.555855,21.619543
2,-95,-10,2021-11-01,2.667682,21.614853
3,-95,-10,2021-11-01,3.856280,21.608278
4,-95,-10,2021-11-01,5.140361,21.601225
...,...,...,...,...,...
477869,-75,-10,2024-08-25,11.405000,15.805074
477870,-75,-10,2024-08-25,9.572997,15.838018
477871,-75,-10,2024-08-25,7.929560,15.865350
477872,-75,-10,2024-08-25,6.440614,15.949759


In [75]:
temp_df.rename

<bound method DataFrame.rename of         lon_rounded_up  lat_rounded_up        date      depth       temp
0                  -95             -10  2021-11-01   0.505760  21.621576
1                  -95             -10  2021-11-01   1.555855  21.619543
2                  -95             -10  2021-11-01   2.667682  21.614853
3                  -95             -10  2021-11-01   3.856280  21.608278
4                  -95             -10  2021-11-01   5.140361  21.601225
...                ...             ...         ...        ...        ...
477869             -75             -10  2024-08-25  11.405000  15.805074
477870             -75             -10  2024-08-25   9.572997  15.838018
477871             -75             -10  2024-08-25   7.929560  15.865350
477872             -75             -10  2024-08-25   6.440614  15.949759
477873             -75             -10  2024-08-25   5.078224  15.973435

[3726006 rows x 5 columns]>

In [76]:
temp_df = temp_df.groupby(
    by=['date', 'lon_rounded_up', 'lat_rounded_up']).mean()
temp_df.reset_index(inplace=True)
temp_df.drop('depth', axis=1, inplace=True)

In [77]:
temp_df

,date,lon_rounded_up,lat_rounded_up,temp
0,2021-11-01,-95,-10,21.560081
1,2021-11-01,-95,-5,22.215484
2,2021-11-01,-95,0,20.370294
3,2021-11-01,-95,5,24.822430
4,2021-11-01,-95,10,25.554212
...,...,...,...,...
205795,2024-08-25,0,15,NaN
205796,2024-08-25,0,20,NaN
205797,2024-08-25,0,25,NaN
205798,2024-08-25,0,30,NaN


In [78]:
temp_df = temp_df[(temp_df.date >= '2021-11-30') & (temp_df.date <= '2024-03-31')]

In [79]:
temp_df

,date,lon_rounded_up,lat_rounded_up,temp
5800,2021-11-30,-95,-10,21.629897
5801,2021-11-30,-95,-5,22.554358
5802,2021-11-30,-95,0,19.667900
5803,2021-11-30,-95,5,23.064395
5804,2021-11-30,-95,10,24.747887
...,...,...,...,...
176395,2024-03-31,0,15,NaN
176396,2024-03-31,0,20,NaN
176397,2024-03-31,0,25,NaN
176398,2024-03-31,0,30,NaN


In [81]:
full_df = pd.merge(o2_df, geo_df, on=['date', 'lon_rounded_up', 'lat_rounded_up'],
                   how='outer')

In [82]:
full_df

,date,lon_rounded_up,lat_rounded_up,o2,chl,no3,po4,si
0,2021-11-30,-95,-10,225.033174,0.149922,2.320183,0.825861,3.871832
1,2021-11-30,-95,-5,221.951588,0.263287,6.655004,1.011894,4.183797
2,2021-11-30,-95,0,185.608996,0.497302,11.343047,1.202757,8.277502
3,2021-11-30,-95,5,191.597901,0.319714,5.631621,0.752967,5.382875
4,2021-11-30,-95,10,183.150434,0.267116,5.082589,0.671990,4.205572
...,...,...,...,...,...,...,...,...
170595,2024-03-31,0,15,NaN,NaN,NaN,NaN,NaN
170596,2024-03-31,0,20,NaN,NaN,NaN,NaN,NaN
170597,2024-03-31,0,25,NaN,NaN,NaN,NaN,NaN
170598,2024-03-31,0,30,NaN,NaN,NaN,NaN,NaN


In [83]:
full_df = pd.merge(full_df, sal_df, on=['date', 'lon_rounded_up', 'lat_rounded_up'],
                   how='outer')

In [84]:
full_df

,date,lon_rounded_up,lat_rounded_up,o2,chl,no3,po4,si,salinity
0,2021-11-30,-95,-10,225.033174,0.149922,2.320183,0.825861,3.871832,35.653211
1,2021-11-30,-95,-5,221.951588,0.263287,6.655004,1.011894,4.183797,35.205782
2,2021-11-30,-95,0,185.608996,0.497302,11.343047,1.202757,8.277502,34.758569
3,2021-11-30,-95,5,191.597901,0.319714,5.631621,0.752967,5.382875,33.841740
4,2021-11-30,-95,10,183.150434,0.267116,5.082589,0.671990,4.205572,33.260691
...,...,...,...,...,...,...,...,...,...
170595,2024-03-31,0,15,NaN,NaN,NaN,NaN,NaN,NaN
170596,2024-03-31,0,20,NaN,NaN,NaN,NaN,NaN,NaN
170597,2024-03-31,0,25,NaN,NaN,NaN,NaN,NaN,NaN
170598,2024-03-31,0,30,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
full_df = pd.merge(full_df, temp_df, on=['date', 'lon_rounded_up', 'lat_rounded_up'],
                   how='outer')

In [86]:
full_df

,date,lon_rounded_up,lat_rounded_up,o2,chl,no3,po4,si,salinity,temp
0,2021-11-30,-95,-10,225.033174,0.149922,2.320183,0.825861,3.871832,35.653211,21.629897
1,2021-11-30,-95,-5,221.951588,0.263287,6.655004,1.011894,4.183797,35.205782,22.554358
2,2021-11-30,-95,0,185.608996,0.497302,11.343047,1.202757,8.277502,34.758569,19.667900
3,2021-11-30,-95,5,191.597901,0.319714,5.631621,0.752967,5.382875,33.841740,23.064395
4,2021-11-30,-95,10,183.150434,0.267116,5.082589,0.671990,4.205572,33.260691,24.747887
...,...,...,...,...,...,...,...,...,...,...
170595,2024-03-31,0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170596,2024-03-31,0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170597,2024-03-31,0,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170598,2024-03-31,0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
full_df.head(500)

,date,lon_rounded_up,lat_rounded_up,o2,chl,no3,po4,si,salinity,temp
0,2021-11-30,-95,-10,225.033174,0.149922,2.320183,0.825861,3.871832,35.653211,21.629897
1,2021-11-30,-95,-5,221.951588,0.263287,6.655004,1.011894,4.183797,35.205782,22.554358
2,2021-11-30,-95,0,185.608996,0.497302,11.343047,1.202757,8.277502,34.758569,19.667900
3,2021-11-30,-95,5,191.597901,0.319714,5.631621,0.752967,5.382875,33.841740,23.064395
4,2021-11-30,-95,10,183.150434,0.267116,5.082589,0.671990,4.205572,33.260691,24.747887
5,2021-11-30,-95,15,193.873248,0.546230,4.377389,0.619023,2.598012,33.654490,25.861815
6,2021-11-30,-95,20,197.810410,0.425429,2.181949,0.358077,2.500064,33.730453,27.840986
7,2021-11-30,-95,25,206.170629,0.207145,0.015289,0.021689,2.099805,36.108274,26.141766
8,2021-11-30,-95,30,211.043761,0.471636,0.023445,0.008767,2.476013,35.588319,24.541881
9,2021-11-30,-95,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
